In [148]:
import pandas as pd
import numpy as np 
import altair as alt 

In [149]:
df = pd.read_csv('744data.csv')
## remove mii rows 
df = df[df.configuration != 'mii']
df.head()

,input,output,model,configuration,machine,tokens_per_second
0,10,10,bloom-560m,ds-1gpu,GCP (T4),31.3
1,10,10,bloom-560m,ds-2gpu,GCP (T4),10.4
2,10,250,bloom-560m,ds-1gpu,GCP (T4),47.5
3,10,250,bloom-560m,ds-2gpu,GCP (T4),15.5
4,10,500,bloom-560m,ds-1gpu,GCP (T4),47.7


We have the following variables:
- input
- output
- model
- configuration
- machine

In [150]:
df_gcp_bloomz = df[(df.model == 'bloomz-560m') & (df.machine == 'GCP (T4)')]
df_gcp_bloomz.head()

,input,output,model,configuration,machine,tokens_per_second


In [151]:
## 1. The effect of deepspeed on the performance
df_csl_bloomz = df[(df.model == 'bloom-560m') & (df.machine == 'CSL (RTX 3090)')].reset_index(drop=True)
df_csl_bloomz.to_csv("data/csl_bloomz.csv", index = False)
df_csl_bloomz

,input,output,model,configuration,machine,tokens_per_second
0,10,10,bloom-560m,no-ds-1gpu,CSL (RTX 3090),0.87
1,10,10,bloom-560m,ds-1gpu,CSL (RTX 3090),0.90
2,10,250,bloom-560m,no-ds-1gpu,CSL (RTX 3090),4.27
3,10,250,bloom-560m,ds-1gpu,CSL (RTX 3090),3.67
4,10,500,bloom-560m,no-ds-1gpu,CSL (RTX 3090),8.47
5,10,500,bloom-560m,ds-1gpu,CSL (RTX 3090),7.20
6,50,10,bloom-560m,no-ds-1gpu,CSL (RTX 3090),0.83
7,50,10,bloom-560m,ds-1gpu,CSL (RTX 3090),0.90
8,50,250,bloom-560m,no-ds-1gpu,CSL (RTX 3090),4.20
9,50,250,bloom-560m,ds-1gpu,CSL (RTX 3090),3.77


## Effect of DeepSpeed on the performance

In [152]:
dff = df_csl_bloomz
# Combining 'output' and 'configuration' into a single column for the X-axis
dff['output_config'] = dff.apply(lambda row: f"{row['output']} ({row['configuration']})", axis=1)

title = alt.TitleParams('Bloom-560M on CSL (NVIDIA GeForce RTX 3090) with single GPU', anchor='middle')
# Altair chart
chart = alt.Chart(dff, title = title).mark_bar().encode(
    x=alt.X('output_config:N', title='Output Size and Configuration', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('tokens_per_second:Q', title='Average Time (in Seconds)'),
    color=alt.Color('configuration:N', legend=alt.Legend(title="Configuration")),
    column=alt.Column('input:N', title='Input Size'),
    tooltip=['input', 'output', 'tokens_per_second', 'configuration'],
).properties(
    width=200,
    height=300
).configure_axis(
    labelFontSize=13,
    titleFontSize=14
).configure_legend(
    titleFontSize=16,
    labelFontSize=16
).configure_title(
    anchor='start',
    fontSize=22
)
chart

alt.Chart(...)

In [153]:
df_colab_t5_large = df[(df.model == 'T5 large') & (df.machine == 'Colab (T4)')].reset_index(drop=True)
df_colab_t5_large.to_csv("data/df_colab_t5_large.csv", index = False)
df_colab_t5_large

,input,output,model,configuration,machine,tokens_per_second
0,10,10,T5 large,no-ds-1gpu,Colab (T4),3.8
1,10,10,T5 large,ds-1gpu,Colab (T4),18.8
2,10,250,T5 large,no-ds-1gpu,Colab (T4),4.2
3,10,250,T5 large,ds-1gpu,Colab (T4),18.8
4,10,500,T5 large,no-ds-1gpu,Colab (T4),4.2
5,10,500,T5 large,ds-1gpu,Colab (T4),17.7
6,50,10,T5 large,no-ds-1gpu,Colab (T4),3.0
7,50,10,T5 large,ds-1gpu,Colab (T4),14.4
8,50,250,T5 large,no-ds-1gpu,Colab (T4),4.9
9,50,250,T5 large,ds-1gpu,Colab (T4),17.6


In [154]:
dff = df_colab_t5_large
# Combining 'output' and 'configuration' into a single column for the X-axis
dff['output_config'] = dff.apply(lambda row: f"{row['output']} ({row['configuration']})", axis=1)

title = alt.TitleParams('T5 Large on Colab (NVIDIA T4) with single GPU', anchor='middle')
# Altair chart
chart = alt.Chart(dff, title = title).mark_bar().encode(
    x=alt.X('output_config:N', title='Output Size and Configuration', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('tokens_per_second:Q', title='Tokens Per Second'),
    color=alt.Color('configuration:N', legend=alt.Legend(title="Configuration")),
    column=alt.Column('input:N', title='Input Size'),
    tooltip=['input', 'output', 'tokens_per_second', 'configuration'],
).properties(
    width=200,
    height=300
).configure_axis(
    labelFontSize=13,
    titleFontSize=14
).configure_legend(
    titleFontSize=16,
    labelFontSize=16
).configure_title(
    anchor='start',
    fontSize=22
)
chart

alt.Chart(...)

### Also checking the effect of input and output

In [155]:
# Assuming 'df' is your DataFrame loaded with the data as described earlier
title = alt.TitleParams('T5 Large on Colab (T4), single GPU', anchor='middle')
interactive_line_chart = alt.Chart(dff, title = title).mark_line(point=True).encode(
    x=alt.X('output:N', title='Output Size'),
    y=alt.Y('tokens_per_second:Q', title='Tokens Per Second'),
    color=alt.Color('configuration:N', legend=alt.Legend(title="Configuration")),
    tooltip=['input', 'output', 'tokens_per_second', 'configuration'],
    column=alt.Column('input:N', title='Input Size')
).interactive().properties(
    width=180,
    height=300
).configure_axis(
    labelFontSize=13,
    titleFontSize=20
).configure_legend(
    titleFontSize=16,
    labelFontSize=16
).configure_title(
    anchor='start',
    fontSize=22
).configure_point(
    size=100
)

# To display the chart in a Jupyter notebook or similar environment
interactive_line_chart.display()

alt.Chart(...)

## Specifically checking the effect of input and output

### 1 gpu vs. 2 gpus

In [156]:
df_gcp_t5_small = df[(df.model == 'T5 small') & (df.machine == 'GCP (T4)')].reset_index(drop=True)
df_gcp_t5_small.to_csv("data/df_gcp_t5_small.csv", index = False)
df_gcp_t5_small.head()

,input,output,model,configuration,machine,tokens_per_second
0,10,10,T5 small,ds-1gpu,GCP (T4),54.4
1,10,10,T5 small,ds-2gpu,GCP (T4),18.2
2,10,250,T5 small,ds-1gpu,GCP (T4),69.7
3,10,250,T5 small,ds-2gpu,GCP (T4),24.9
4,10,500,T5 small,ds-1gpu,GCP (T4),73.7


In [157]:
# Assuming 'df' is your DataFrame loaded with the data as described earlier
title = alt.TitleParams('T5 Small on Google Cloud Platform (NVIDIA T4)', anchor='middle')

interactive_line_chart = alt.Chart(df_gcp_t5_small, title = title).mark_line(point=True).encode(
    x=alt.X('output:N', title='Output Size'),
    y=alt.Y('tokens_per_second:Q', title='Tokens Per Second'),
    color=alt.Color('configuration:N', legend=alt.Legend(title="Configuration")),
    tooltip=['input', 'output', 'tokens_per_second', 'configuration'],
    column=alt.Column('input:N', title='Input Size')
).interactive().properties(
    width=180,
    height=300
).configure_axis(
    labelFontSize=13,
    titleFontSize=20
).configure_legend(
    titleFontSize=16,
    labelFontSize=16
).configure_title(
    anchor='start',
    fontSize=22
).configure_point(
    size=100
)

# To display the chart in a Jupyter notebook or similar environment
interactive_line_chart.display()

alt.Chart(...)

In [158]:
df_gcp_t5_large = df[(df.model == 'T5 large') & (df.machine == 'GCP (T4)')].reset_index(drop=True)
df_gcp_t5_large.to_csv("data/df_gcp_t5_large.csv", index = False)
df_gcp_t5_large.head()

,input,output,model,configuration,machine,tokens_per_second
0,10,10,T5 large,ds-1gpu,GCP (T4),53.7
1,10,10,T5 large,ds-2gpu,GCP (T4),17.5
2,10,250,T5 large,ds-1gpu,GCP (T4),74.5
3,10,250,T5 large,ds-2gpu,GCP (T4),24.6
4,10,500,T5 large,ds-1gpu,GCP (T4),76.0


In [159]:
# Assuming 'df' is your DataFrame loaded with the data as described earlier
title = alt.TitleParams('T5 Large on Google Cloud Platform (NVIDIA T4)', anchor='middle')

interactive_line_chart = alt.Chart(df_gcp_t5_large, title = title).mark_line(point=True).encode(
    x=alt.X('output:N', title='Output Size'),
    y=alt.Y('tokens_per_second:Q', title='Tokens Per Second'),
    color=alt.Color('configuration:N', legend=alt.Legend(title="Configuration")),
    tooltip=['input', 'output', 'tokens_per_second', 'configuration'],
    column=alt.Column('input:N', title='Input Size')
).interactive().properties(
    width=180,
    height=300
).configure_axis(
    labelFontSize=13,
    titleFontSize=20
).configure_legend(
    titleFontSize=16,
    labelFontSize=16
).configure_title(
    anchor='start',
    fontSize=22
).configure_point(
    size=100
)

# To display the chart in a Jupyter notebook or similar environment
interactive_line_chart.display()

alt.Chart(...)

## The effect of Model on performance

In [160]:
df_gcp_2gpus = df[(df.configuration == 'ds-2gpu') & (df.machine == 'GCP (T4)')].reset_index(drop=True)
df_gcp_2gpus.head()

,input,output,model,configuration,machine,tokens_per_second
0,10,10,bloom-560m,ds-2gpu,GCP (T4),10.4
1,10,250,bloom-560m,ds-2gpu,GCP (T4),15.5
2,10,500,bloom-560m,ds-2gpu,GCP (T4),16.1
3,50,10,bloom-560m,ds-2gpu,GCP (T4),15.6
4,50,250,bloom-560m,ds-2gpu,GCP (T4),17.7


In [161]:
# Assuming 'df' is your DataFrame loaded with the data as described earlier
title = alt.TitleParams('Comparing different models on Google Cloud Platform (T4) with 2 GPUs', anchor='middle')

interactive_line_chart = alt.Chart(df_gcp_2gpus, title = title).mark_line(point=True).encode(
    x=alt.X('output:N', title='Output Size'),
    y=alt.Y('tokens_per_second:Q', title='Tokens Per Second'),
    color=alt.Color('model:N', legend=alt.Legend(title="Languae model")),
    tooltip=['input', 'output', 'tokens_per_second', 'model'],
    column=alt.Column('input:N', title='Input Size')
).interactive().properties(
    width=180,
    height=300
).configure_axis(
    labelFontSize=13,
    titleFontSize=20
).configure_legend(
    titleFontSize=16,
    labelFontSize=16
).configure_title(
    anchor='start',
    fontSize=22
).configure_point(
    size=100
)

# To display the chart in a Jupyter notebook or similar environment
interactive_line_chart.display()

alt.Chart(...)

In [162]:
df_gcp_1gpu = df[(df.configuration == 'ds-1gpu') & (df.machine == 'GCP (T4)')].reset_index(drop=True)
df_gcp_1gpu.head()

,input,output,model,configuration,machine,tokens_per_second
0,10,10,bloom-560m,ds-1gpu,GCP (T4),31.3
1,10,250,bloom-560m,ds-1gpu,GCP (T4),47.5
2,10,500,bloom-560m,ds-1gpu,GCP (T4),47.7
3,50,10,bloom-560m,ds-1gpu,GCP (T4),48.9
4,50,250,bloom-560m,ds-1gpu,GCP (T4),54.4


In [163]:
# Assuming 'df' is your DataFrame loaded with the data as described earlier
title = alt.TitleParams('Comparing different models on Google Cloud Platform (T4) with 1 GPU', anchor='middle')

interactive_line_chart = alt.Chart(df_gcp_1gpu, title = title).mark_line(point=True).encode(
    x=alt.X('output:N', title='Output Size'),
    y=alt.Y('tokens_per_second:Q', title='Tokens Per Second'),
    color=alt.Color('model:N', legend=alt.Legend(title="Languae model")),
    tooltip=['input', 'output', 'tokens_per_second', 'model'],
    column=alt.Column('input:N', title='Input Size')
).interactive().properties(
    width=180,
    height=300
).configure_axis(
    labelFontSize=13,
    titleFontSize=20
).configure_legend(
    titleFontSize=16,
    labelFontSize=16
).configure_title(
    anchor='start',
    fontSize=22
).configure_point(
    size=100
)

# To display the chart in a Jupyter notebook or similar environment
interactive_line_chart.display()

alt.Chart(...)

## Comparing different machines

In [164]:
df_t5_large_1gpu = df[(df.configuration == 'ds-1gpu') & (df.model == 'T5 large')].reset_index(drop=True)
df_t5_large_1gpu.head()

,input,output,model,configuration,machine,tokens_per_second
0,10,10,T5 large,ds-1gpu,GCP (T4),53.7
1,10,250,T5 large,ds-1gpu,GCP (T4),74.5
2,10,500,T5 large,ds-1gpu,GCP (T4),76.0
3,50,10,T5 large,ds-1gpu,GCP (T4),66.5
4,50,250,T5 large,ds-1gpu,GCP (T4),82.6


In [165]:
title = alt.TitleParams('Comparing different machines on the model of T5 Large on 1 GPU', anchor='middle')

interactive_line_chart = alt.Chart(df_t5_large_1gpu, title = title).mark_line(point=True).encode(
    x=alt.X('output:N', title='Output Size'),
    y=alt.Y('tokens_per_second:Q', title='Tokens Per Second'),
    color=alt.Color('machine:N', legend=alt.Legend(title="Machine")),
    tooltip=['input', 'output', 'tokens_per_second', 'machine'],
    column=alt.Column('input:N', title='Input Size')
).interactive().properties(
    width=180,
    height=300
).configure_axis(
    labelFontSize=13,
    titleFontSize=20
).configure_legend(
    titleFontSize=16,
    labelFontSize=16
).configure_title(
    anchor='start',
    fontSize=22
).configure_point(
    size=100
)

# To display the chart in a Jupyter notebook or similar environment
interactive_line_chart.display()

alt.Chart(...)